In [1]:
import pandas as pd
from glob import glob

In [2]:
april_files = glob('./Raw/raw dump April2016/*All*')
april_files

['./Raw/raw dump April2016/All complaints during 2012.xls',
 './Raw/raw dump April2016/All complaints during 2013.xls',
 './Raw/raw dump April2016/All complaints during 2014.xls',
 './Raw/raw dump April2016/All complaints during 2015.xls',
 './Raw/raw dump April2016/All complaints during 2016YTD.xls']

In [3]:
df_list = [pd.read_excel(file) for file in april_files]
april_df = pd.concat(df_list, ignore_index=True)

In [11]:
april_df.columns = [col.replace(' ', '_') for col in april_df.columns]
sorted(april_df.columns.tolist())

['Accsued_Officer_FName',
 'AccusedOfficer_FName',
 'Accused_Appointment_Date',
 'Accused_Detail',
 'Accused_Officer_FName',
 'Accused_Officer_LName',
 'Accused_Position',
 'Accused_Star',
 'Accused_Unit',
 'Address',
 'Beat',
 'CURRENT_CATEGORY',
 'CURRENT_CATEGORY_CODE',
 'CURRENT_STATUS',
 'Complaint_Number',
 'Discipline_Code',
 'FINDING_CODE',
 'INITIAL_CATEGORY',
 'INITIAL_CATEGORY_CODE',
 'IPRA_Closed_Date',
 'IPRA_Investigator_FirstName',
 'IPRA_Investigator_LastName',
 'Incident_Time_End',
 'Incident_Time_Start',
 'Involved_Officer_Age',
 'Involved_Officer_Detail',
 'Involved_Officer_FName',
 'Involved_Officer_Lname',
 'Involved_Officer_Position',
 'Involved_Officer_Race',
 'Involved_Officer_Sex',
 'Involved_Officer_Unit',
 'LOCATION_CODE',
 'Notification_Date',
 'Recommended_Number_Of_Days',
 'Street']

In April, several of there are multiple fname columns due to misspellings of the column headers. These can be combined, as there are no cases where one column has conflicting information with the other column

In [15]:
# confirming there are no conflicting values
sub_df = april_df[['Accused_Officer_FName', 'AccusedOfficer_FName']]
sub_df = sub_df[(~sub_df.Accused_Officer_FName.isnull()) & 
                 (~sub_df.AccusedOfficer_FName.isnull())]
print(sum(sub_df.Accused_Officer_FName != sub_df.AccusedOfficer_FName))

sub_df = april_df[['Accused_Officer_FName', 'Accsued_Officer_FName']]
sub_df = sub_df[(~sub_df.Accused_Officer_FName.isnull()) & 
                (~sub_df.Accsued_Officer_FName.isnull())]
print(sum(sub_df.Accused_Officer_FName != sub_df.Accsued_Officer_FName))

sub_df = april_df[['Accsued_Officer_FName', 'AccusedOfficer_FName']]
sub_df = sub_df[(~sub_df.Accsued_Officer_FName.isnull()) & 
                (~sub_df.AccusedOfficer_FName.isnull())]
print(sum(sub_df.Accsued_Officer_FName != sub_df.AccusedOfficer_FName))

0
0
0


In [16]:
april_df.Accused_Officer_FName.fillna(april_df.Accsued_Officer_FName, inplace=True)
april_df.Accused_Officer_FName.fillna(april_df.AccusedOfficer_FName, inplace=True)
april_df.drop(['Accsued_Officer_FName', 'AccusedOfficer_FName'], axis=1, inplace=True)

In [17]:
sorted(april_df.columns)

['Accused_Appointment_Date',
 'Accused_Detail',
 'Accused_Officer_FName',
 'Accused_Officer_LName',
 'Accused_Position',
 'Accused_Star',
 'Accused_Unit',
 'Address',
 'Beat',
 'CURRENT_CATEGORY',
 'CURRENT_CATEGORY_CODE',
 'CURRENT_STATUS',
 'Complaint_Number',
 'Discipline_Code',
 'FINDING_CODE',
 'INITIAL_CATEGORY',
 'INITIAL_CATEGORY_CODE',
 'IPRA_Closed_Date',
 'IPRA_Investigator_FirstName',
 'IPRA_Investigator_LastName',
 'Incident_Time_End',
 'Incident_Time_Start',
 'Involved_Officer_Age',
 'Involved_Officer_Detail',
 'Involved_Officer_FName',
 'Involved_Officer_Lname',
 'Involved_Officer_Position',
 'Involved_Officer_Race',
 'Involved_Officer_Sex',
 'Involved_Officer_Unit',
 'LOCATION_CODE',
 'Notification_Date',
 'Recommended_Number_Of_Days',
 'Street']

In [19]:
april_df.to_csv('./Clean/April2016/april2016.csv')
april_df.to_excel('./Clean/April2016/april2016.xlsx')

In [40]:
summary = pd.DataFrame({yr: {'Row Count': len(df), 'Unique CRID Count': len(df.Complaint_Number.dropna().unique())} 
              for df, yr in zip(df_list, range(2012, 2017))})
summary = summary.transpose()

In [41]:
summary

,Row Count,Unique CRID Count
2012,48293,1877
2013,39561,1913
2014,32454,1663
2015,14727,1392
2016,1720,330


In [46]:
categories = sorted(april_df.INITIAL_CATEGORY.dropna().unique().tolist())

In [69]:
initial = april_df.groupby('INITIAL_CATEGORY').apply(lambda x: len(x.Complaint_Number.dropna().unique())).reset_index()

In [70]:
current = april_df.groupby('CURRENT_CATEGORY').apply(lambda x: len(x.Complaint_Number.dropna().unique())).reset_index()

In [78]:
initial.columns = ['Category', 'INITIAL_CATEGORY_COUNT']
current.columns = ['Category', 'CURRENT_CATEGORY_COUNT']

In [77]:
[cat for cat in current.CURRENT_CATEGORY if cat not in initial.INITIAL_CATEOGRY.tolist()]

['CONDUCT UNBECOMING',
 'ELBOW STRIKE',
 'FAILURE TO IDENTIFY',
 'IMPROPER SERVICE OF WARRANT',
 'MISUSE OF DEPARTMENT EQUIPMENT / SUPPLIES',
 'PROPERTY - FAILED TO INVENTORY',
 'TASER (LASER TARGETED)']

In [80]:
category_summary = pd.merge(initial, current, on='Category', how='outer')

In [82]:
category_summary.fillna(0, inplace=True)

In [88]:
category_summary.INITIAL_CATEGORY_COUNT = category_summary.INITIAL_CATEGORY_COUNT.astype(int)
category_summary.CURRENT_CATEGORY_COUNT = category_summary.CURRENT_CATEGORY_COUNT.astype(int)

In [90]:
category_summary.to_csv('/home/dan/Desktop/category_summary.csv')